In [642]:
from bs4 import BeautifulSoup
import requests
import re
import csv
from datetime import datetime

import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [643]:
class FaroesProperties:
    def __init__(self, website=None,
                 address=None, houseNum=None,
                 city=None, postNum=None,
                 price=None,LatesPrice=None, validDate=None,
                 date=None, buildingSize=None,
                 landSize=None, room=None,
                 floor=None):
        if website is None:
            self.websites = "None"
        self.websites = website
                     
        if address is None:
            self.addresses = "None"
        self.addresses = address
                     
        if houseNum is None:
            self.houseNums = "None"
        self.houseNums = houseNum

        if city is None:
            self.cities = "None"
        self.cities = city

        if postNum is None:
            self.postNums = "None"
        self.postNums = postNum
                     
        if prices is None:
            self.prices = "None"
        self.prices = price
                     
        if LatestPrices is None:
            self.LatestPrices = "None"
        self.LatestPrices = LatesPrice
                     
        if validDates is None:
            self.validDates = "None"
        self.validDates = validDate
                     
        if dates is None:
            self.dates = "None"
        self.dates = date
                     
        if buildingSizes is None:
            self.buildingSizes = "None"
        self.buildingSizes = buildingSize
                     
        if landSize is None:
            self.landSizes = "None"
        self.landSizes = landSize
                     
        if rooms is None:
            self.rooms = "None"
        self.rooms = room
                     
        if floors is None:
            self.floors = "None"
        self.floors = floor
                     
    def display(self):
        print("Heimasída: ", self.websites, 
              "\nAddressa: ", self.addresses,
              "\nHúsnummar: ",self.houseNums,
              "\nPostnr: ", self.postNums,
              "\nBýur: ", self.cities,
              "\nPrísur: ", self.prices,
              "\nSeinasti bod: ",self.LatestPrices, 
              "\nGaldandi til dato: ", self.validDates,
              "\nDato: ", self.dates,
              "\nm2 í húsinum: ", self.buildingSizes,"\nm2 á økinum: ", self.landSizes,
              "\nRúm: ", self.rooms,
              "\nHæddir: ", self.floors, 
              "\n")
    def writeToCSV(self):
        print("test")

In [644]:
betriURL = 'https://www.betriheim.fo/'
def scrape_page(driver):
    url = f"{betriURL}"
    driver.get(url)
    time.sleep(2)  # Give the page time to load

    return driver.page_source

# Starts up chrome
driver = webdriver.Chrome()  
try:  
    betriResponse = scrape_page(driver)
finally:
    driver.quit()    


# betriResponse = requests.get(betriURL)
betriSoup = BeautifulSoup(betriResponse, 'html.parser')

skynURL = 'https://www.skyn.fo/ognir-til-soelu'
skynResponse = requests.get(skynURL)
skynSoup = BeautifulSoup(skynResponse.text, 'html.parser')

properties = []

In [645]:
BetriPropertyWrappers = betriSoup.find_all('article', class_="c-property c-card grid")

In [646]:
def CheckValueText(property, attribute, classVal):
    value = property.find(attribute, class_=classVal)
    if value is not None:
        value = value.text
    else:
        value = None
    return value

In [647]:
def addressParse(addresses):
    if str(addresses):
        pattern1 = r"((\w+ )(\w+$))"
        pattern2 = r"(\w+[ ]?[^0-9]+[ ]?[^0-9]+)"
        pattern3 = r"([0-9]+\w?)"
        match1 = re.search(pattern1, addresses)
        match2 = re.search(pattern2, addresses)
        match3 = re.search(pattern3, addresses)
        city = match1.group(3)
        postNum = match1.group(2)
        address = match2.group(1)
        houseNum = match3.group(1)
        yield address 
        yield houseNum 
        yield city 
        yield postNum
    

In [648]:
def priceParse(prices):
    if prices:
        pattern = r"(Kr. )"
        match = re.sub(pattern, "", prices)
        pattern = r"(\.)"
        match = re.sub(pattern, "", match)
        return int(match)

In [649]:
def latestPriceOfferParse(latestPriceOffer):
    if latestPriceOffer:
        pattern = r"(Seinasta boð: Kr. )"
        match = re.sub(pattern, "", latestPriceOffer)
        pattern = r"(\.)"
        match = re.sub(pattern, "", match)
        return int(match)

In [650]:
def convert_to_date(last_two_words):
    if last_two_words:
        try:
            # Assuming the last two words are in the format "dd-mm-yyyy, kl HH:MM"
            date_obj = datetime.strptime(last_two_words, "%d-%m-%Y, kl %H:%M")
            return date_obj
        except ValueError:
            return "Invalid date format"

def priceOfferValidDateParse(priceOfferValidDate):
    if priceOfferValidDate:
        pattern = r"(Galdandi til )"
        match = re.sub(pattern, "", priceOfferValidDate)
        formatted_date = convert_to_date(match)
        return formatted_date

In [651]:
def convertToDigits(yearBuilt):
    if yearBuilt:
        pattern = r"([0-9]+)"
        formatted_year = re.search(pattern, yearBuilt)
        # pattern = r"()"
        # formatted_year = re.sub(pattern, "", match)
        return int(formatted_year.group(1))

In [653]:
for property in BetriPropertyWrappers:
    websites = "Betri"
    addresses = CheckValueText(property, 'address', "medium")
    addressEntries = addressParse(addresses)
    address = next(addressEntries)
    houseNum = next(addressEntries)
    city = next(addressEntries)
    postNum = next(addressEntries)
    prices = CheckValueText(property, 'div', "price")
    priceEntries = int(priceParse(prices))
    latestPriceOffer = CheckValueText(property, 'div', "latest-offer")
    latestPriceOfferEntries = latestPriceOfferParse(latestPriceOffer)
    priceOfferValidDate = CheckValueText(property, 'div', "valid")
    priceOfferValidDateEntry = priceOfferValidDateParse(priceOfferValidDate)
    # print(priceOfferValidDateEntry)
    yearBuilt = CheckValueText(property, 'div', "date")
    yearBuiltEntry = convertToDigits(yearBuilt)
    insideM2 = CheckValueText(property, 'div',"building-size")
    insideM2Entry = convertToDigits(insideM2)
    outsideM2 = CheckValueText(property, 'div', "land-size")
    outsideM2Entry = convertToDigits(outsideM2)
    rooms = CheckValueText(property, 'div', "rooms")
    roomEntry = convertToDigits(rooms)
    floors = CheckValueText(property, 'div', "floors")
    floorEntry = convertToDigits(floors)
    # print(address, "-",houseNum,"-",city,"-",postNum)
    print(address,"-",houseNum,"-",postNum,"-",city,"-",priceEntries,"-"
              ,latestPriceOfferEntries,"-",priceOfferValidDateEntry,"-",yearBuiltEntry
              ,"-",insideM2Entry,"-",outsideM2Entry,"-",roomEntry,"-",floorEntry)

    prop = FaroesProperties(websites, address, houseNum, city, postNum,
                            priceEntries,latestPriceOfferEntries, priceOfferValidDateEntry,yearBuiltEntry,insideM2Entry,
                            outsideM2Entry,roomEntry,floorEntry)
    properties.append(prop)


Heimaratún  - 3 - 100  - Tórshavn - 3595000 - None - None - 1959 - 173 - 137 - None - 4
Gerðisbreyt  - 3 - 176  - Velbastaður - 2995000 - 3020000 - 2023-10-03 14:00:00 - 1985 - 154 - 602 - 4 - 2
Grásteinsgøta  - 2B - 100  - Tórshavn - 4700000 - None - None - 2023 - 174 - 305 - 5 - 2
Kráarvegur  - 20 - 530  - Fuglafjørður - 2095000 - None - None - 1990 - 264 - 500 - 5 - 2
Liðabrekkan  - 15 - 380  - Sørvágur - 975000 - None - None - 1918 - 772 - 173 - 3 - 3
Steigarbrekka  - 14 - 360  - Sandavágur - 1500000 - 1200000 - 2023-09-11 11:00:00 - 1926 - 104 - 179 - None - 3
Garðavegur  - 12 - 240  - Skopun - 1350000 - None - None - 1924 - 156 - 137 - 4 - 3
Stoffalág  - 59 - 100  - Tórshavn - 1795000 - None - None - None - None - 424 - None - None
Garðavegur  - 70 - 700  - Klaksvík - 695000 - None - None - 1940 - 150 - 150 - 5 - 3
Bøgøta  - 30 - 900  - Vágur - 1690000 - None - None - 1960 - 134 - 382 - None - 2
Karlamagnusarbreyt  - 44 - 188  - Hoyvík - 2995000 - None - None - 2021 - 83 - None -

In [635]:
# for property in BetriPropertyWrappers:
#     websites = "Betri"
#     addresses = CheckValueText(property, 'address', "medium")
#     prices = CheckValueText(property, 'div', "price")
#     latestPriceOffer = CheckValueText(property, 'div', "latest-offer")
#     priceOfferValidDate = CheckValueText(property, 'div', "valid")
#     yearBuilt = CheckValueText(property, 'div', "date")
#     insideM2 = CheckValueText(property, 'div',"building-size")
#     outsideM2 = CheckValueText(property, 'div', "land-size")
#     rooms = CheckValueText(property, 'div', "rooms")
#     floors = CheckValueText(property, 'div', "floors")
#     prop = FaroesProperties(websites, addresses,prices,LatestPrices,
#                             validDates,dates,buildingSizes,
#                             landSizes,rooms,floors)
#     properties.append(prop)

In [654]:
for prop in properties:
    prop.display()

Heimasída:  Betri 
Addressa:  Heimaratún  
Húsnummar:  3 
Postnr:  100  
Býur:  Tórshavn 
Prísur:  3595000 
Seinasti bod:  None 
Galdandi til dato:  None 
Dato:  1959 
m2 í húsinum:  173 
m2 á økinum:  137 
Rúm:  None 
Hæddir:  4 

Heimasída:  Betri 
Addressa:  Gerðisbreyt  
Húsnummar:  3 
Postnr:  176  
Býur:  Velbastaður 
Prísur:  2995000 
Seinasti bod:  3020000 
Galdandi til dato:  2023-10-03 14:00:00 
Dato:  1985 
m2 í húsinum:  154 
m2 á økinum:  602 
Rúm:  4 
Hæddir:  2 

Heimasída:  Betri 
Addressa:  Grásteinsgøta  
Húsnummar:  2B 
Postnr:  100  
Býur:  Tórshavn 
Prísur:  4700000 
Seinasti bod:  None 
Galdandi til dato:  None 
Dato:  2023 
m2 í húsinum:  174 
m2 á økinum:  305 
Rúm:  5 
Hæddir:  2 

Heimasída:  Betri 
Addressa:  Kráarvegur  
Húsnummar:  20 
Postnr:  530  
Býur:  Fuglafjørður 
Prísur:  2095000 
Seinasti bod:  None 
Galdandi til dato:  None 
Dato:  1990 
m2 í húsinum:  264 
m2 á økinum:  500 
Rúm:  5 
Hæddir:  2 

Heimasída:  Betri 
Addressa:  Liðabrekkan  
Húsnum